# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
# Import API key
from api_keys import geoapify_key



8b8ee97b15f84910a79c95072e24b901


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,3.63,66,71,1.34,US,1732491558
1,1,yellowknife,62.4560,-114.3525,-9.21,93,100,3.09,CA,1732491781
2,2,tolanaro,-25.0319,46.9987,25.04,88,40,3.09,MG,1732491781
3,3,hamilton,39.1834,-84.5333,11.60,69,0,3.60,US,1732490948
4,4,grytviken,-54.2811,-36.5092,4.57,72,3,5.06,GS,1732491773


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
citydata_humidity= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha= 0.7,
    frame_width = 700,
    frame_height = 500,
    hover_cols= ["City","Humidity"]
    
    
    
)


# Display the map
citydata_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values - max temp range 21 to 27, wind speed less than 4.5 m/s and cloudiness = 0
idealcitydf= city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] <27) \
                                   & (city_data_df["Wind Speed"] <4.5) \
                                   & (city_data_df["Cloudiness"] ==0)]

# Drop any rows with null values
idealcitydf= idealcitydf.dropna()

# Display sample data

idealcitydf.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
83,83,kedougou,12.5579,-12.1743,26.32,47,0,0.00,SN,1732491794
229,229,sur,22.5667,59.5289,24.57,61,0,4.03,OM,1732491817
253,253,ciudad del carmen,18.6333,-91.8333,26.02,50,0,2.06,MX,1732491821
258,258,victorica,-36.2151,-65.4359,23.34,32,0,3.95,AR,1732491822
260,260,brady,31.1352,-99.3351,25.25,38,0,2.57,US,1732491823


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealcitydf[["City","Lat","Lng","Country","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] =""

# Display sample data
print(len(hotel_df))
hotel_df

9


,City,Lat,Lng,Country,Humidity,Hotel Name
83,kedougou,12.5579,-12.1743,SN,47,
229,sur,22.5667,59.5289,OM,61,
253,ciudad del carmen,18.6333,-91.8333,MX,50,
258,victorica,-36.2151,-65.4359,AR,32,
260,brady,31.1352,-99.3351,US,38,
329,chabahar,25.2919,60.6430,IR,59,
379,kidal,18.4411,1.4078,ML,15,
400,rustaq,23.3908,57.4244,OM,44,
411,luanda,-8.8368,13.2343,AO,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":1
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude= row["Lat"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
#filter any hotels that do not have a hotel found
hotel_df= hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df

Starting hotel search
kedougou - nearest hotel: Auberge Thomas Sankara
sur - nearest hotel: Sur Hotel
ciudad del carmen - nearest hotel: Holiday Inn Express
victorica - nearest hotel: Hotel Mazamet
brady - nearest hotel: Holiday Inn Express and Suites
chabahar - nearest hotel: هتل سپیده
rustaq - nearest hotel: Shoumoukh Tourist Hotel
luanda - nearest hotel: Hotel Forum


,City,Lat,Lng,Country,Humidity,Hotel Name
83,kedougou,12.5579,-12.1743,SN,47,Auberge Thomas Sankara
229,sur,22.5667,59.5289,OM,61,Sur Hotel
253,ciudad del carmen,18.6333,-91.8333,MX,50,Holiday Inn Express
258,victorica,-36.2151,-65.4359,AR,32,Hotel Mazamet
260,brady,31.1352,-99.3351,US,38,Holiday Inn Express and Suites
329,chabahar,25.2919,60.6430,IR,59,هتل سپیده
400,rustaq,23.3908,57.4244,OM,44,Shoumoukh Tourist Hotel
411,luanda,-8.8368,13.2343,AO,83,Hotel Forum


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
#%%capture --no-display


# Configure the map plot
hotelplot= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Max Temp",
    scale = 1,
    color = "City",
    alpha= 0.7,
    frame_width = 700,
    frame_height = 500,
    hover_cols= ["City","Hotel Name", "Country"]
)


# Display the map
hotelplot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)